# Run with Data Collection

This notebook combines collision avoidance (CA) and steering models with data collection. Any time the JetBot gets confused, save an image to enhance the dataset.

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import PIL.Image
import cv2
import numpy as np
import traitlets
from IPython.display import display
import ipywidgets
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg
from jetbot import Robot
import time
import os

# Alexnet model for Collision Avoidance (CA)
model_CA = torchvision.models.alexnet(pretrained=False)
model_CA.classifier[6] = torch.nn.Linear(model_CA.classifier[6].in_features, 2)

# Resnet model for steering
model_Steer = torchvision.models.resnet18(pretrained=False)
model_Steer.fc = torch.nn.Linear(512, 2)

# Load the trained weights model for Collision Avoidance
model_CA.load_state_dict(torch.load('best_model_FB.pth'))

# Load the trained weights model for Steering
model_Steer.load_state_dict(torch.load('best_steering_model_xy.pth'))

# Load the weights on the GPU
device = torch.device('cuda')
model_CA = model_CA.to(device)
model_Steer = model_Steer.to(device)
model_Steer = model_Steer.eval().half()


# Begin image preprocessing for Collision Avoidance
mean_CA = 255.0 * np.array([0.485, 0.456, 0.406])
stdev_CA = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean_CA, stdev_CA)

def preprocess_CA(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

# Begin image preprocessing for Steering
mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess_Steer(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]
# End image preprocessing

# Begin camera instantiation with memory optimizations
camera = Camera.instance(width=300, height=300, fps=10)

image_widget = ipywidgets.Image()

traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

#display(image_widget)

# Instantiate JetBot controls
from jetbot import Robot
robot = Robot()

# Steering PD control sliders
speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, description='Speed Gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.2, description='Steering Gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.0, description='Steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='Steering Bias')
display(speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)

# Sliders to show steering predictions
x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='Steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='Speed')
blocked_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description = 'Blocked')
display(ipywidgets.HBox([y_slider, speed_slider]))
display(x_slider, steering_slider)

angle = 0.0
angle_last = 0.0

# Update the Steering model for each camera change
def update(change):
    # Global variables 
    global blocked_slider, robot, angle, angle_last
    
    # Current image
    image = change['new'] 
    
    # Begin image processing for CA 
    CA_input = preprocess_CA(image)
    CA_output = model_CA(CA_input)
    
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    CA_output = F.softmax(CA_output, dim=1)
    
    prob_blocked = float(CA_output.flatten()[0])
    prob_free = float(CA_output.flatten()[1])
    
    blocked_slider.value = prob_blocked
    # End CA image processing
      
    
    # End Steering image processing
    
    # Begin robot behavior
    if prob_blocked > 0.9:
        robot.backward(0.5)
        time.sleep(0.1)
    
    if prob_free > 0.9:
        robot.forward(0.4)
       
    else:
        # Begin image processing for steering
        xy = model_Steer(preprocess_Steer(image)).detach().float().cpu().numpy().flatten()
        x = xy[0]
        y = (0.5 - xy[1]) / 2.0

        x_slider.value = x
        y_slider.value = y
        speed_slider.value = speed_gain_slider.value

        angle = np.arctan2(x, y)

        pid = (angle * steering_gain_slider.value) + ((angle - angle_last) * steering_dgain_slider.value)

        angle_last = angle                                            

        steering_slider.value = pid + steering_bias_slider.value
        robot.left_motor.value = max(min(speed_slider.value + steering_slider.value, 1.0), 0.0)
        robot.right_motor.value = max(min(speed_slider.value - steering_slider.value, 1.0), 0.0)
    
update({'new': camera.value}) # Update Jetbot with the current image

# Current Parameters:
# Speed Gain:    0.00
# Steering Gain: 0.20
# Steering kd:   0.05
# Steering Bias:-0.05

FloatSlider(value=0.0, description='Speed Gain', max=1.0, step=0.01)

FloatSlider(value=0.2, description='Steering Gain', max=1.0, step=0.01)

FloatSlider(value=0.0, description='Steering kd', max=0.5, step=0.001)

FloatSlider(value=0.0, description='Steering Bias', max=0.3, min=-0.3, step=0.01)

FloatSlider(value=0.0, description='x', max=1.0, min=-1.0)

FloatSlider(value=0.0, description='Steering', max=1.0, min=-1.0)

In [2]:
# this attaches the 'update' function to the 'value' traitlet of our camera
camera.observe(update, names='value')  

Stop the JetBot:

In [3]:
import time

camera.unobserve(update, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

robot.stop()

#camera.stop() # Free up the camera for the next run